In [1]:
from script import *
import re

In [2]:
print(file_name)
excel_file = pd.ExcelFile(file_name)
fle = pd.read_excel(file_name)
print(excel_file.sheet_names)
df = pd.DataFrame(fle)
# print(df)

2023_Kwara_P3B.xlsx
['Guidelines - READ ME', 'Hypotheses', 'List of population', 'STATE SUMMARY', '1. ASA', '2. BARUTEN', '3. EDU', '4. EKITI', '5. IFELODUN', '6. ILORIN EAST', '7. ILORIN SOUTH ', '8. ILORIN WEST', '9. IREPODUN', '10. ISIN', '11. KAIAMA', '12. MORO', '13. OFFA', '14. OKE ERO', '15. OYUN', '16. PATIGI', 'Additional information', 'LGA store location']


In [3]:
file = "2021_Ogun_P3B.xlsx"
sheets = get_sheets(file)
print(sheets)

['1. ABEOKUTA NORTH', '10. IKENNE', '11. IMEKO AFON', '12. IPOKIA', '13. OBAFEMI OWODE', '14. ODEDA', '15. ODOGBOLU', '16. OGUN WATERSIDE', '17. REMO NORTH', '18. SAGAMU', '19. YEWA NORTH']


In [82]:


for sheet in sheets:
    demo_df = pd.read_excel(file,sheet_name=sheet)
    row_index=actual_header_row(demo_df)
    clean_data = remove_first_blank_column(file,row_index,sheet)
    new_data = remove_unwanted_columns(clean_data,["Wards","List of contiguous communities/ settlements","Population\n(2021)"])
    base_data = remove_blank_wards_rows(new_data)
    subtotal_df = base_data
    wards_df = base_data
    # print(base_data["Wards"])
    for indx in range(len(base_data)):
        if base_data["Wards"][indx] == "sub total":
            wards_df = wards_df.drop(indx)
        else:
            subtotal_df = subtotal_df.drop(indx)
    wards_df=wards_df.reset_index(drop=True)
    subtotal_df=subtotal_df.reset_index(drop=True)

   
    for  indx in range(len(wards_df)):
        # wards_df["Wards"][indx]=subtotal_df["Wards"][indx]
        wards_df["List of contiguous communities/ settlements"][indx]=subtotal_df["List of contiguous communities/ settlements"][indx]
        wards_df["Population\n(2021)"][indx]= subtotal_df["Population\n(2021)"][indx]
    wards_df = wards_df.sort_values(by='Population\n(2021)')
    wards_df["Cumulative frequency"] = wards_df['Population\n(2021)'].cumsum()
    sum_population = wards_df['Population\n(2021)'].sum()
    population_interval = sum_population//8
    start_population = random.choice(range(1000,20000,273))
    pop_list = []
    for indx in range(8):
        pop_list.append(start_population)
        start_population = start_population+population_interval

    # print(sum_population,population_interval,start_population)
    # print(pop_list)
    print(wards_df["Cumulative frequency"], wards_df["Wards"])

    final_df = pd.DataFrame({"Wards":[],"Total communities":[],"Population":[],"Cumulative frequency":[],"Clusters":[]})
    for i, population in enumerate(pop_list):
        print(i, population)
        for indx in range(len(wards_df)):
            if population <= wards_df["Cumulative frequency"][indx]:
                ward = wards_df["Wards"][indx]
                com = wards_df["List of contiguous communities/ settlements"][indx]
                pop = wards_df["Population\n(2021)"][indx]
                cum = wards_df["Cumulative frequency"][indx]
                cluster = f"Cluster {i+1} ({population})"
                data = {"Wards":[ward],"Total communities":[com],"Population":[pop],"Cumulative frequency":[cum],"Clusters":[cluster]}
                new_row= pd.DataFrame(data)
                final_df = pd.concat([final_df, new_row], ignore_index=True)
                # print(data)
                break 
            continue     

    print(final_df)
    break

0         Wards List of contiguous communities/ settlements  \
14     TOTORO 3                                          16   
2   IBEREKODO 3                                          14   
0   IBEREKODO 1                                          14   
1   IBEREKODO 2                                          10   
15    TOTORO  4                                          13   
12     TOTORO 1                                           9   
10       SABO 1                                          20   
13     TOTORO 2                                          20   
6     IKEREKU 1                                          53   
7     IKEREKU 2                                          59   
3   IBEREKODO 4                                          11   
9      OLORUNDA                                          41   
4   IBEREKODO 5                                          15   
8         IMALA                                          80   
11       SABO 2                                        